In [1]:
import pandas as pd
import numpy as np
import re
import os

pd.set_option("display.max_columns", None)
pd.set_option('display.max_colwidth', None)

In [2]:
pdfs = pd.read_csv('data-pdf.csv')
pdfs.head()

,pdf,teléfono,habitaciones,Equipment,precio_alquiler,superficie,baños,contacto
0,"Alquiler de piso en avenida de la Constitución, 5, San Miguel, Murcia.pdf",690 964 656,3,Amueblado,800.0,110.0,2.0,Elena
1,"Alquiler de piso en avenida de la Constitución, 5, San Miguel, Murcia.pdf",651 915 502,3,Amueblado,850.0,125.0,2.0,Marco
2,"Alquiler de piso en calle Riquelme, 14, Santa Catalina-San Bartolomé, Murcia.pdf",644 463 987,4,equipada,300.0,80.0,1.0,NaN
3,"Alquiler de piso en calle Cánovas del Castillo, 67, Santiago de la Ribera, San Javier.pdf",665 795 659,2,amueblado,800.0,82.0,2.0,Angel
4,"Alquiler de piso en calle urb veneziola, 3, Playa del Esparto-Veneziola, La Manga del Mar Menor.pdf",671 305 483,2,Amueblado,300.0,28.0,1.0,Alicia


In [3]:
pdfs = pdfs.rename(columns={'contacto': 'nombre', 
                            'Equipment': 'amueblado', 
                            'precio_alquiler': 'precio',
                            'teléfono': 'telefono',
                            'baños': 'banos',
                            })
pdfs['tipo_transaccion'] = 'Alquiler'
pdfs['perfil'] = 'Particular'
pdfs['fecha'] = '2018-01-01'
pdfs['precio'] = pdfs['precio'].fillna(0).astype(int)
pdfs['superficie'] = pdfs['superficie'].fillna(0).astype(int)
pdfs['banos'] = pdfs['banos'].fillna(0).astype(int)
# Cambiar datos concretos de una columna, por ejemplo, cambiar el valor de 'precio' donde 'ciudad' es 'Murcia'
# Reemplazar los valores de 'No especificado' en la columna 'habitaciones' por 0
pdfs.loc[pdfs['habitaciones'] == 'No especificado', 'habitaciones'] = 0
# Cambiar el tipo de la columna 'habitaciones' a entero
pdfs['habitaciones'] = pdfs['habitaciones'].astype(int)



# Definir la función para extraer los datos
def extraer_datos(row):
    # Usar una expresión regular para capturar "tipo de vivienda", "street" y "place"
    match = re.search(r'Alquiler de (.+?) en (.*?, \d+), (.*?)\.pdf', row)
    if match:
        tipo = match.group(1)  # Captura el tipo de vivienda (piso, casa, etc.)
        calle = match.group(2)         # Captura la calle y número
        barrio = match.group(3)          # Captura el barrio y la ciudad
        return pd.Series([tipo, calle, barrio])
    else:
        return pd.Series([None, None, None])  # Si no coincide, devuelve None

# Aplicar la función al DataFrame y crear las nuevas columnas
pdfs[['tipo', 'calle', 'barrio']] = pdfs['pdf'].apply(extraer_datos)

In [5]:
euspay = pd.read_csv('euspay-murcia.csv')
euspay.drop("Unnamed: 0", axis=1, inplace=True)
euspay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11137 entries, 0 to 11136
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              11137 non-null  int64 
 1   name_contact    11009 non-null  object
 2   telephone       10869 non-null  object
 3   type_operation  11137 non-null  object
 4   property        11137 non-null  object
 5   profile         11137 non-null  object
 6   street          9329 non-null   object
 7   place           11137 non-null  object
 8   meters          11137 non-null  int64 
 9   rooms           11137 non-null  object
 10  orientation     6050 non-null   object
 11  features        10621 non-null  object
 12  price           11137 non-null  int64 
 13  source          11137 non-null  object
 14  id_source       11137 non-null  int64 
 15  url_source      11137 non-null  object
 16  date            11137 non-null  object
 17  date_scrape     11137 non-null  object
 18  ask_ad

In [6]:
euspay = euspay.rename(columns={'id_source': 'id_inmueble',
                                'name_contact': 'nombre',
                                'telephone': 'telefono',
                                'type_operation': 'tipo_transaccion',
                                'property': 'tipo',
                                'profile': 'perfil',
                                'street': 'calle',
                                'place': 'barrio',
                                'meters': 'superficie',
                                'rooms': 'habitaciones',
                                'orientation': 'orientacion',
                                'price': 'precio',
                                'date_scrape': 'fecha',
                                })

euspay = euspay.drop(columns={'id', 'ask_advertiser', 'date', 'source'})

In [7]:
euspay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11137 entries, 0 to 11136
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   nombre            11009 non-null  object
 1   telefono          10869 non-null  object
 2   tipo_transaccion  11137 non-null  object
 3   tipo              11137 non-null  object
 4   perfil            11137 non-null  object
 5   calle             9329 non-null   object
 6   barrio            11137 non-null  object
 7   superficie        11137 non-null  int64 
 8   habitaciones      11137 non-null  object
 9   orientacion       6050 non-null   object
 10  features          10621 non-null  object
 11  precio            11137 non-null  int64 
 12  id_inmueble       11137 non-null  int64 
 13  url_source        11137 non-null  object
 14  fecha             11137 non-null  object
dtypes: int64(3), object(12)
memory usage: 1.3+ MB


In [ ]:
bd_pdf = pdfs[['nombre', 'perfil', 'telefono', 'tipo_transaccion', 'tipo', 'calle', 'barrio', 'superficie', 'habitaciones', 'banos',  ]]


In [8]:
pdfs.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1970 entries, 0 to 1969
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   pdf               1970 non-null   object
 1   telefono          1970 non-null   object
 2   habitaciones      1970 non-null   int64 
 3   amueblado         1695 non-null   object
 4   precio            1970 non-null   int64 
 5   superficie        1970 non-null   int64 
 6   banos             1970 non-null   int64 
 7   nombre            1251 non-null   object
 8   tipo_transaccion  1970 non-null   object
 9   perfil            1970 non-null   object
 10  fecha             1970 non-null   object
 11  tipo              1645 non-null   object
 12  calle             1645 non-null   object
 13  barrio            1645 non-null   object
dtypes: int64(4), object(10)
memory usage: 215.6+ KB


In [ ]:
df_merged = pd.merge(pdfs, euspay, on='id_inmueble')